# Hypothesis Testing

For this assignment, I will be working with the [US Health Insurance Dataset](https://docs.google.com/spreadsheets/d/e/2PACX-1vQBN8DPW2rdiRrY34eEM53HAzakNGSRrw4ogI-j8HyCUrbqTB_z4CeIn2IvjLF-w_6sOe5pIlypJGAA/pub?output=csv) from [Kaggle](https://www.kaggle.com/teertha/ushealthinsurancedataset).

Use the hypothesis testing skills to answer the following questions:

- Q1. Do smokers have higher insurance charges than non-smokers?
- Q2. Are men more likely to smoke than women?
- Q3. Do different regions have different charges, on average?

In [2]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [4]:
! pip install pandas

In [6]:
data = 'Data/insurance - insurance.csv'
df = pd.read_csv(data)
df.head(2)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.90,0,yes,southwest,16884.9240
1,18,male,33.77,1,no,southeast,1725.5523


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


## 1. Do smokers have higher insurance charges than non-smokers?

### 1. State Null Hypothesis and Alternative Hypothesis

- $H_0$ (Null Hypothesis):
- $H_A$ (Alternative Hypothesis):  

### 2. Select the correct test according to the data type and number of samples

### 3. Test the assumptions of your selected test.

### 4. Execute the selected test, or the alternative test (if you do not meet the assumptions)

### 5. Interpret your p-value and reject or fail to reject your null hypothesis 

### 6. Show a supporting visualization that helps display the result

## 2. Are men more likely to smoke than women?

- $H_0$ (Null Hypothesis):
- $H_A$ (Alternative Hypothesis):  

### 1. State Null Hypothesis and Alternative Hypothesis

### 2. Select the correct test according to the data type and number of samples

### 3. Test the assumptions of your selected test.

### 4. Execute the selected test, or the alternative test (if you do not meet the assumptions)

### 5. Interpret your p-value and reject or fail to reject your null hypothesis 

### 6. Show a supporting visualization that helps display the result

## 3. Do different regions have different charges, on average?

### 1. State Null Hypothesis and Alternative Hypothesis

- $H_0$ (Null Hypothesis):
- $H_A$ (Alternative Hypothesis):  

### 2. Select the correct test according to the data type and number of samples

### 3. Test the assumptions of your selected test.

### 4. Execute the selected test, or the alternative test (if you do not meet the assumptions)

### 5. Interpret your p-value and reject or fail to reject your null hypothesis 

### 6. Show a supporting visualization that helps display the result